In [ ]:
import pandas as pd
import datetime
import numpy as np
import matplotlib.pyplot as plt

### reading in crisis1

In [ ]:
#https://stackoverflow.com/questions/26521266/using-pandas-to-pd-read-excel-for-multiple-worksheets-of-the-same-workbook
crisis1 = pd.ExcelFile('../data/NSSCrisisv.1.xlsx')

In [ ]:
crisis1.sheet_names

In [ ]:
crisis1 = crisis1.parse('in')

In [ ]:
crisis1.head(20)

In [ ]:
crisis1.info()

### cleaning up crisis1

#drop columns with ALL NaN values
#https://datatofish.com/drop-columns-with-nan/
crisis1 = crisis1.dropna(axis=1, how ='all')
crisis1 = crisis1.dropna(axis=0, how='all')
crisis1.head()

In [ ]:
#trim all whitespace
#https://stackoverflow.com/questions/33788913/pythonic-efficient-way-to-strip-whitespace-from-every-pandas-data-frame-cell-tha
crisis1 = crisis1.apply(lambda x: x.str.strip() if x.dtype == "str" else x)

In [ ]:
#replace column names with row1 values
#https://www.adamsmith.haus/python/answers/how-to-convert-a-pandas-dataframe-row-to-column-headers-in-python
header_row = 1
crisis1.columns = crisis1.iloc[header_row]
crisis1.head()

In [ ]:
crisis1 = crisis1.drop(header_row)
crisis1 = crisis1.reset_index(drop=True)
crisis1.head()

In [ ]:
crisis1.tail()

### reading in crisis2

In [ ]:
crisis2 = pd.ExcelFile('../data/NSSCrisisv.2.xlsx')

In [ ]:
crisis2.sheet_names

In [ ]:
crisis2 = crisis2.parse('in')

In [ ]:
crisis2.head(20)

In [ ]:
crisis2.tail()

In [ ]:
crisis2.info()

### cleaning up crisis2

In [ ]:
#drop columns with ALL NaN values
#https://datatofish.com/drop-columns-with-nan/
crisis2 = crisis2.dropna(axis=1, how ='all')
crisis2 = crisis2.dropna(axis=0, how='all')
crisis2.head()

In [ ]:
#trim all whitespace
#https://stackoverflow.com/questions/33788913/pythonic-efficient-way-to-strip-whitespace-from-every-pandas-data-frame-cell-tha
crisis2 = crisis2.apply(lambda x: x.str.strip() if x.dtype == "str" else x)

In [ ]:
#replace column names with row1 values
#https://www.adamsmith.haus/python/answers/how-to-convert-a-pandas-dataframe-row-to-column-headers-in-python
crisis2.columns = crisis2.iloc[header_row]
crisis2 = crisis2.drop(header_row)
crisis2 = crisis2.reset_index(drop=True)
crisis2.head()

### reading in call_volume

In [ ]:
#https://stackoverflow.com/questions/26521266/using-pandas-to-pd-read-excel-for-multiple-worksheets-of-the-same-workbook
call_volume = pd.ExcelFile('../data/Contact_center_call_volume_2020-2022.xlsx')
call_volume.sheet_names

In [ ]:
call_volume2020 = call_volume.parse('2020')

In [ ]:
call_volume2021 = call_volume.parse('2021')

In [ ]:
call_volume2022 = call_volume.parse('2022')

In [ ]:
call_volume2020.head()

In [ ]:
call_volume2021.head(20)

### combining crisis1 and crisis2

In [ ]:
crisis = pd.concat([crisis2, crisis1])

In [ ]:
crisis.info()

In [ ]:
crisis = crisis.reset_index(drop=True)
crisis.head(20)

In [ ]:
crisis.tail(20)

In [ ]:
crisis.shape

## 1. Examine call volume and identify surge times, seasons, or events.

### Ideas: 2021 vs 2022, avg call volume per month, avg call volume per day of the. month

In [ ]:
crisis['CallDateAndTimeStart'].dtype

In [ ]:
print(type(crisis['CallDateAndTimeStart']))

In [ ]:
#this really won't work since the time is included - let's get a column w/ just the date
crisis['CallDateAndTimeStart'].value_counts()

In [ ]:
#this gave a weird result: crisis['CallDateAndTimeStart'] = str(crisis['CallDateAndTimeStart'])
#maybe due to python versions? solution pieced together from here: https://stackoverflow.com/questions/22231592/pandas-change-data-type-of-series-to-string
crisis['CallDateAndTimeStart'] = crisis['CallDateAndTimeStart'].astype('str')

In [ ]:
crisis['call_date'] = crisis.CallDateAndTimeStart.str[0:10]

In [ ]:
crisis.head(30)

In [ ]:
#trying again with just the date (new column we just created)
#let's make this a dataframe for analysis
call_dates = crisis['call_date'].value_counts()
call_dates = pd.DataFrame(call_dates).reset_index()
call_dates.columns = ['date', 'number_calls']
call_dates.head(50)

In [ ]:
call_dates.describe()

In [ ]:
#going to order call_dates by date so that when we plot it, we can see trends over time
call_dates = call_dates.sort_values(by='date')
call_dates.head()

In [ ]:
call_dates.plot(x="date", y="number_calls", kind="line")
plt.show()

In [ ]:
#clearly that's way too many data points, so let's look at the top 50 days
#call_dates_top50 = 

## 2. Analyze trends in call content by looking for which issues are most frequent and what is the average count of issues per call.

In [ ]:
#first, let's try to isolate columns with "CRISIS" in the title
#(based on README, that's what we're looking for)
#https://towardsdatascience.com/interesting-ways-to-select-pandas-dataframe-columns-b29b82bbfb33
#(at that link, see section on selecting columns based on substring)

crisis_issues = crisis.loc[:,['CRISIS Issues' in i for i in crisis.columns]]
crisis_issues.head()

In [ ]:
#how many non-NAN values per column? and make it a dataframe
crisis_notes = crisis_issues.count().to_frame().reset_index()

#rename columns
crisis_notes.columns = ['crisis_issue', 'call_volume']

#sort values by call_volume
crisis_notes = crisis_notes.sort_values(by='call_volume', ascending=False)

crisis_notes.head(20)